In [1]:
import numpy as np
import pandas as pd
import os

datadir = os.path.join('..', 'WISDM_at_v2.0')
raw_headers = ['user','activity','time','x','y','z']
act_headers = ['user','X0','X1','X2','X3','X4','X5','X6','X7','X8','X9','Y0','Y1','Y2','Y3','Y4','Y5','Y6','Y7','Y8','Y9','Z0','Z1','Z2','Z3','Z4','Z5','Z6','Z7','Z8','Z9','XAVG','YAVG','ZAVG','XPEAK','YPEAK','ZPEAK','XABSOLDEV', 'YABSOLDEV','ZABSOLDEV','XSTANDDEV','YSTANDDEV','ZSTANDDEV','Resultant','Activity']


In [2]:
labelled_act = os.path.join(datadir,'WISDM_at_v2.0_transformed.csv')
labelled_raw = os.path.join(datadir, 'WISDM_at_v2.0_raw.txt')
raw_data = pd.read_csv(labelled_raw, header=None, names = raw_headers, parse_dates=True)
raw_data = raw_data.loc[raw_data['time']>0]
raw_data['time'] = pd.to_datetime(raw_data['time'], unit = 'ms', errors='coerce')
act_data = pd.read_csv(labelled_act, header=None, names = act_headers)

C:\ProgramData\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
time_user_raw = raw_data[['user','time']]
user_duration = time_user_raw.groupby('user').agg({'time':[min,max]})
user_duration.columns = ["_".join(x) for x in user_duration.columns.ravel()]
user_duration['duration'] = user_duration['time_max'] - user_duration['time_min']
print user_duration.sort_values(by='duration').loc[act_data.user.unique()]['duration']

user
194       51 days 02:50:35.887000
998        0 days 00:36:51.205000
1097                          NaT
1104       0 days 00:26:29.013000
1117      22 days 22:12:28.932000
1205       0 days 00:01:39.789000
1238      75 days 21:24:22.851000
1246       1 days 09:49:00.316000
1247       0 days 00:03:59.996000
1253       2 days 15:50:43.069000
1269       0 days 00:00:19.903000
1274       0 days 00:04:07.093000
1276       0 days 00:00:39.903000
1277       1 days 04:24:11.038000
1280       0 days 00:02:08.213000
1319     161 days 19:42:39.082000
1320      16 days 00:39:44.952000
1477       0 days 00:12:35.201000
1480       0 days 00:02:50.623000
1491       0 days 00:02:28.380000
1511       0 days 00:01:09.845000
1512       0 days 00:50:59.809000
1518       0 days 00:04:36.221000
1531       0 days 00:18:12.781000
1554       0 days 01:15:45.781000
1559       0 days 00:10:11.107000
1603   12612 days 03:36:08.153000
1676       0 days 00:33:22.581000
1679       0 days 00:00:50.151000
1683     

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """


In [18]:
activity_duration = {'Walking': 0, 'Jogging': 1, 'Standing': 2, 'Sitting': 3, 'Stairs': 4, 'LyingDown': 5}
user_feats = []
users = []
for userid in act_data.user.unique():
    user_act_data = act_data.loc[act_data['user']==userid]['Activity']
    prev = user_act_data.iloc[0]
    #feat = [walk_dur, jog_dur, stand_dur, sit_dur, stair_dur, ly_dur, no_tx, walk_jog, walk_sit, sit_walk]
    feat = [0.0]*10
    tot_dur = 0
    for i,activity in user_act_data.iteritems():
        feat_index = activity_duration.get(activity)
        feat[feat_index] = feat[feat_index] + 1
        if activity!= prev:
            feat[6] = feat[6] + 1
            if prev=='Walking' and activity=='Jogging':
                feat[7] = feat[7] + 1
            elif prev=='Walking' and activity=='Sitting':
                feat[8] = feat[8] + 1
            elif prev=='Sitting' and activity=='Walking':
                feat[9] = feat[9] + 1
        prev = activity
        tot_dur = tot_dur + 1
    ufeat = [x/tot_dur for x in feat]
    user_feats.append(ufeat)
    users.append(userid)

In [19]:
print users
print user_feats

[194, 998, 1097, 1104, 1117, 1205, 1238, 1246, 1247, 1253, 1269, 1274, 1276, 1277, 1280, 1319, 1320, 1477, 1480, 1491, 1511, 1512, 1518, 1531, 1554, 1559, 1603, 1676, 1679, 1683, 1696, 1703, 1707, 1723, 1724, 1726, 1742, 1745, 1750, 1757, 1758, 1759, 1761, 1763, 1774, 1775, 1778, 1793, 1797, 1799, 1802, 1809, 1813, 1814]
[[0.18235294117647058, 0.11176470588235295, 0.38235294117647056, 0.07647058823529412, 0.23529411764705882, 0.011764705882352941, 0.047058823529411764, 0.0058823529411764705, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.39908256880733944, 0.13302752293577982, 0.25229357798165136, 0.10091743119266056, 0.10550458715596331, 0.009174311926605505, 0.03669724770642202, 0.0045871559633027525, 0.0, 0.0], [0.0, 0.0, 0.5310344827586206, 0.2482758620689655, 0.1310344827586207, 0.0896551724137931, 0.027586206896551724, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.3409090909090909, 0.0

In [45]:
for userid in raw_data.user.unique():
    user_raw_data = raw_data.loc[raw_data['user']==userid]
    prev_time = user_raw_data.iloc[0]['time']
    gap = 0
    for index, attr in user_raw_data.iterrows():
        if (attr[2] - prev_time).total_seconds() > 0.7:
            gap = gap + 1
        prev_time = attr[2]
    print userid, gap


1679 0
599 8
685 5
669 9
1277 5
674 0
594 5
678 0
648 6
584 0
1733 2
582 0
1769 3
636 0
1758 3
708 0
711 0
687 0
563 0
621 3
623 3
720 1
1793 4
568 0
640 250
1715 0
671 0
694 0
664 3
585 8
684 0
662 1
1480 0
655 0
1511 0
1603 29
651 0
579 7
613 0
590 0
639 0
587 0
719 3
635 0
1742 6
624 0
710 4
676 0
1761 3
693 5
1696 7
646 5
713 3
610 0
615 5
705 0
653 0
702 5
604 0
1702 0
618 0
654 5
656 0
606 5
998 3
586 0
1501 0
1319 8
668 0
1768 0
1100 4
1205 0
573 1
1491 1
712 0
1518 2
730 1
622 3
647 0
1727 1
1477 1
588 3
634 1
661 0
686 0
690 0
709 0
663 1
597 1
1656 5
1683 0
630 1
1724 2
1649 0
616 0
691 0
625 0
612 0
650 8
1753 3
658 0
1750 4
598 7
729 1
714 5
607 0
628 0
589 0
728 0
925 1
1638 0
1713 1
593 0
600 5
637 0
641 7
609 1
633 13
1745 0
1783 3
1770 1
688 1
1269 0
1276 0
1246 2
1707 5
605 0
1247 1
695 2
595 0
602 7
194 15
583 0
716 0
727 1
1797 4
1320 7
1676 13
673 0
1802 2
611 7
1117 8
689 0
1757 0
1774 5
666 6
617 0
706 0
627 0
675 2
1799 3
1703 7
670 0
1719 3
1759 6
1554 4
1778 4


In [44]:
rraw_data = pd.read_csv(labelled_raw, header=None, names = raw_headers, parse_dates=True)
user_raw_data = rraw_data.loc[rraw_data['user']==1097]
user_raw_data
#act_data.loc[act_data['user']==1097]

,user,activity,time,x,y,z
